[1757. Recyclable and Low Fat Products](https://leetcode.com/problems/recyclable-and-low-fat-products/?envType=study-plan-v2&envId=top-sql-50)

PANDAS

In [0]:
import pandas as pd

data = [['0', 'Y', 'N'], ['1', 'Y', 'Y'], ['2', 'N', 'Y'], ['3', 'Y', 'Y'], ['4', 'N', 'N']]
products = pd.DataFrame(data, columns=['product_id', 'low_fats', 'recyclable']).astype(
    {'product_id': 'int64', 'low_fats': 'category', 'recyclable': 'category'})

In [0]:
products.head()

,product_id,low_fats,recyclable
0,0,Y,N
1,1,Y,Y
2,2,N,Y
3,3,Y,Y
4,4,N,N


In [0]:
filtered_data = products[(products["low_fats"]=="Y") & (products["recyclable"]=="Y")]

In [0]:
print(filtered_data[["product_id"]].to_string(index=False))

 product_id
          1
          3


SPARK

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

spark_product = spark.createDataFrame(products)
spark_product.printSchema()

root
 |-- product_id: long (nullable = true)
 |-- low_fats: string (nullable = true)
 |-- recyclable: string (nullable = true)



In [0]:
spark_product.show()

+----------+--------+----------+
|product_id|low_fats|recyclable|
+----------+--------+----------+
|         0|       Y|         N|
|         1|       Y|         Y|
|         2|       N|         Y|
|         3|       Y|         Y|
|         4|       N|         N|
+----------+--------+----------+



In [0]:
import pyspark.sql.functions as F

spark_product.filter((F.col('low_fats')=="Y")&(F.col('recyclable')=="Y")).select('product_id').show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



SQL

In [0]:
spark_product.createOrReplaceTempView('Products')
spark.sql(''' SELECT product_id From Products 
          Where low_fats="Y"and recyclable="Y"
          ''').show()

+----------+
|product_id|
+----------+
|         1|
|         3|
+----------+



[584. Find Customer Referee](https://leetcode.com/problems/find-customer-referee/description/?envType=study-plan-v2&envId=top-sql-50)

PANDAS

In [0]:
data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype(
    {'id': 'Int64', 'name': 'object', 'referee_id': 'Int64'})

In [0]:
customer

,id,name,referee_id
0,1,Will,<NA>
1,2,Jane,<NA>
2,3,Alex,2
3,4,Bill,<NA>
4,5,Zack,1
5,6,Mark,2


In [0]:
referee_customer = customer[(customer['referee_id'].isna())]

In [0]:
print(referee_customer[["name"]].to_string(index=False))

name
Will
Jane
Bill


PYSPARK

In [0]:
# spark has issues with null values when using Pandas
from pyspark.sql.types import *
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("reference_id", IntegerType(), True)
])

In [0]:
customer_df = spark.createDataFrame(customer, schema=schema)
customer_df.show()


+---+----+------------+
| id|name|reference_id|
+---+----+------------+
|  1|Will|        null|
|  2|Jane|        null|
|  3|Alex|           2|
|  4|Bill|        null|
|  5|Zack|           1|
|  6|Mark|           2|
+---+----+------------+



In [0]:
customer_df.filter((F.col("reference_id").isNull())).select("name").show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
+----+



SQL

In [0]:
customer_df.createOrReplaceTempView('customer')
spark.sql('''
          SELECT name from customer WHERE reference_id is null;
          ''').show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
+----+



[595. Big Countries](https://leetcode.com/problems/big-countries/description/?envType=study-plan-v2&envId=top-sql-50)

Pandas

In [0]:


data = [['Afghanistan', 'Asia', 652230, 25500100, 20343000000], ['Albania', 'Europe', 28748, 2831741, 12960000000],
        ['Algeria', 'Africa', 2381741, 37100000, 188681000000], ['Andorra', 'Europe', 468, 78115, 3712000000],
        ['Angola', 'Africa', 1246700, 20609294, 100990000000]]
world = pd.DataFrame(data, columns=['name', 'continent', 'area', 'population', 'gdp']).astype(
    {'name': 'object', 'continent': 'object', 'area': 'Int64', 'population': 'Int64', 'gdp': 'Int64'})

In [0]:
world

,name,continent,area,population,gdp
0,Afghanistan,Asia,652230,25500100,20343000000
1,Albania,Europe,28748,2831741,12960000000
2,Algeria,Africa,2381741,37100000,188681000000
3,Andorra,Europe,468,78115,3712000000
4,Angola,Africa,1246700,20609294,100990000000


In [0]:
filtered_world = world[(world['area'] >= 3000000) | (world["population"] >= 25000000)]

In [0]:
print(filtered_world[["name","population","area"]].to_string(index=False))

       name  population    area
Afghanistan    25500100  652230
    Algeria    37100000 2381741


Pyspark

In [0]:
world_df = spark.createDataFrame(world)

In [0]:
world_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- area: long (nullable = true)
 |-- population: long (nullable = true)
 |-- gdp: long (nullable = true)



In [0]:
world_df.filter((F.col('area')>=3000000) | (F.col('population')>=25000000)).select("name","population","area").show()

+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



SQL

In [0]:
world_df.createOrReplaceTempView('world')

spark.sql(
    '''
    select name,population, area
    From world where area>=3000000 or population>=25000000;
    '''
).show()

+-----------+----------+-------+
|       name|population|   area|
+-----------+----------+-------+
|Afghanistan|  25500100| 652230|
|    Algeria|  37100000|2381741|
+-----------+----------+-------+



[1148. Article Views I](https://leetcode.com/problems/big-countries/description/?envType=study-plan-v2&envId=top-sql-50)

Pandas

In [0]:


data = [[1, 3, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})

In [0]:
views

,article_id,author_id,viewer_id,view_date
0,1,3,5,2019-08-01
1,1,3,6,2019-08-02
2,2,7,7,2019-08-01
3,2,7,6,2019-08-02
4,4,7,1,2019-07-22
5,3,4,4,2019-07-21
6,3,4,4,2019-07-21


In [0]:
filter_view = views[(views['author_id'] == views['viewer_id'])]

In [0]:
print(filter_view[['author_id']].drop_duplicates().sort_values('author_id').to_string(index=False))

 author_id
         4
         7


Pyspark

In [0]:
views_df = spark.createDataFrame(views)

In [0]:
views_df.printSchema()

root
 |-- article_id: long (nullable = true)
 |-- author_id: long (nullable = true)
 |-- viewer_id: long (nullable = true)
 |-- view_date: timestamp (nullable = true)



In [0]:
views_df.filter(F.col('author_id')==F.col('viewer_id')).select(F.col('author_id').alias('id')).distinct().orderBy('id').show()

+---+
| id|
+---+
|  4|
|  7|
+---+



SQL

In [0]:
views_df.createOrReplaceTempView('views')

spark.sql('''
select distinct author_id as id
from views
where author_id=viewer_id
order by id;
''').show()

+---+
| id|
+---+
|  4|
|  7|
+---+



[1683. Invalid Tweets](https://leetcode.com/problems/invalid-tweets/description/?envType=study-plan-v2&envId=top-sql-50)

Pandas

In [0]:


data = [[1, 'Vote for Biden'], [2, 'Let us make America great again!']]
tweets = pd.DataFrame(data, columns=['tweet_id', 'content']).astype({'tweet_id':'Int64', 'content':'object'})

In [0]:
tweets

,tweet_id,content
0,1,Vote for Biden
1,2,Let us make America great again!


In [0]:
tweets['length_content'] = tweets['content'].apply(len)

In [0]:
tweets.columns

Out[99]: Index(['tweet_id', 'content', 'length_content'], dtype='object')

In [0]:
tweets

,tweet_id,content,length_content
0,1,Vote for Biden,14
1,2,Let us make America great again!,32


In [0]:
filter_tweets = tweets[tweets['length_content']>=15]

In [0]:
print(filter_tweets[["tweet_id"]].to_string(index=False))

 tweet_id
        2


Pyspark

In [0]:
tweets_df = spark.createDataFrame(tweets)

In [0]:
tweets_df.filter(F.length(F.col('content'))>15).select('tweet_id').show()

+--------+
|tweet_id|
+--------+
|       2|
+--------+



SQL

In [0]:
tweets_df.createOrReplaceTempView('tweets')

spark.sql('''
select tweet_id from tweets where length(content)>15;
''').show()

+--------+
|tweet_id|
+--------+
|       2|
+--------+

